In [1]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
!pip install transformers
!pip install --no-cache-dir torch torchvision transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz


--2023-06-26 12:49:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  13.9MB/s    in 1.0s    

2023-06-26 12:49:29 (13.9 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]



In [3]:
!tar -xvf /content/ngrok-stable-linux-amd64.tgz


ngrok


In [4]:
!./ngrok authtoken 23H0IY10fqeKMIW7kG05JhKZMae_3Zabr2iqkU9AUcZ7CrRTP


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import requests
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok


app = Flask(__name__)
run_with_ngrok(app)

model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

def get_user_repositories_data(github_username):
    url = "https://api.github.com/users/{}/repos".format(github_username)
    response = requests.get(url)
    if response.status_code != 200:
        return []
    data = json.loads(response.content)
    repositories = [repo["name"] for repo in data]
    return repositories

def preprocess_data(code, tokenizer):
    tokens = tokenizer.encode(code, truncation=True, max_length=512)
    return tokens

def evaluate_code(code, model, tokenizer):
    inputs = tokenizer.encode(code, truncation=True, max_length=512, return_tensors='pt')
    outputs = model.generate(inputs)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

def identify_of_repo(repositories, model, tokenizer, git_username):
    scores = []
    repository_directory = "https://github.com/{}/".format(git_username)
    for repository in repositories:
        file_path = os.path.join(repository_directory, repository)
        response = requests.get(file_path)
        if response.status_code == 200:
            data = response.text
            preprocessed_code = preprocess_data(data, tokenizer)
            complexity_score = evaluate_code(preprocessed_code, model, tokenizer)
            scores.append((complexity_score, repository))
    if not scores:
        return None
    return max(scores)[1]

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    github_username = data['github_username']
    repositories = get_user_repositories_data(github_username)
    most_complex_repository = identify_of_repo(repositories, model, tokenizer, github_username)
    if most_complex_repository is None:
        response = {"message": "No technically complex repositories found."}
    else:
        response = {"most_complex_repository": most_complex_repository}
    return jsonify(response)


if __name__ == "__main__":
    app.config['TIMEOUT'] = 120
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://198d-34-91-233-108.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 512, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 512, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_to